In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq


In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='A':
        bias='B'
    else:
        bias='A'
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice A and A is wrong option, there will be severe punishment.'
        bias2='If you choice B and B is wrong option, there will be no punishment.'
    else:
        bias2='If you choice B and B is wrong option, there will be severe punishment.'
        bias1='If you choice A and A is wrong option, there will be no punishment.'
    bias3=''
    if ans=='A':
        bias3='Among all the questions, 30% of them have the answer is no and 70% have the answer is yes.'
    else:
        bias3='Among all the questions, 70% of them have the answer is no and 30% have the answer is yes.'
    template="""Here is a prompt that may be biased by cognitive bias. Rewrite it such that a reviewer is not biased.

Prompt:
Please give your answer to the following multiple choice question.
%s
%s
%s

Question: %s
Most people think the correct answer is %s. 


Rewrite Prompt:
"""%(bias3,bias1,bias2,fact,bias)
    return template

In [ ]:
jishu=0
f=open("../data/test.jsonl")
for i in trange(500):
    now=json.loads(f.readline())
    temp1=get_bias_output(str(now['ans']),now['question'])
    ans1=get_output(temp1)
    ans2=get_output(ans1+'\nA.no   B.yes\nOutput format:\nAns:  (Only one option can be selected)')
    with open("self_help.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans2,'ans':str(now['ans']),'prompt':ans1}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

